In [ ]:
! apt-get install -y g++ openjdk-8-jdk python3-dev curl git build-essential
! pip install konlpy "tweepy<4.0.0"
! /bin/bash <(curl -s https://raw.githubusercontent.com/konlpy/konlpy/master/scripts/mecab.sh)

In [2]:
import torch
from torch.nn.utils import clip_grad_norm_
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torch.utils.tensorboard import SummaryWriter

from tqdm import tqdm

import ko_mrc.datasets as datasets
import ko_mrc.utils as utils
import ko_mrc.models as models

In [3]:
dataset = datasets.KoMRC.load('data/train.json')
print("Number of Questions", len(dataset))
print(dataset[0])

Number of Questions 12037
{'guid': '798db07f0b9046759deed9d4a35ce31e', 'context': '올여름 장마가 17일 제주도에서 시작됐다. 서울 등 중부지방은 예년보다 사나흘 정도 늦은 이달 말께 장마가 시작될 전망이다.17일 기상청에 따르면 제주도 남쪽 먼바다에 있는 장마전선의 영향으로 이날 제주도 산간 및 내륙지역에 호우주의보가 내려지면서 곳곳에 100㎜에 육박하는 많은 비가 내렸다. 제주의 장마는 평년보다 2~3일, 지난해보다는 하루 일찍 시작됐다. 장마는 고온다습한 북태평양 기단과 한랭 습윤한 오호츠크해 기단이 만나 형성되는 장마전선에서 내리는 비를 뜻한다.장마전선은 18일 제주도 먼 남쪽 해상으로 내려갔다가 20일께 다시 북상해 전남 남해안까지 영향을 줄 것으로 보인다. 이에 따라 20~21일 남부지방에도 예년보다 사흘 정도 장마가 일찍 찾아올 전망이다. 그러나 장마전선을 밀어올리는 북태평양 고기압 세력이 약해 서울 등 중부지방은 평년보다 사나흘가량 늦은 이달 말부터 장마가 시작될 것이라는 게 기상청의 설명이다. 장마전선은 이후 한 달가량 한반도 중남부를 오르내리며 곳곳에 비를 뿌릴 전망이다. 최근 30년간 평균치에 따르면 중부지방의 장마 시작일은 6월24~25일이었으며 장마기간은 32일, 강수일수는 17.2일이었다.기상청은 올해 장마기간의 평균 강수량이 350~400㎜로 평년과 비슷하거나 적을 것으로 내다봤다. 브라질 월드컵 한국과 러시아의 경기가 열리는 18일 오전 서울은 대체로 구름이 많이 끼지만 비는 오지 않을 것으로 예상돼 거리 응원에는 지장이 없을 전망이다.', 'question': '북태평양 기단과 오호츠크해 기단이 만나 국내에 머무르는 기간은?', 'answers': [{'text': '한 달가량', 'answer_start': 478}, {'text': '한 달', 'answer_start': 478}]}


In [4]:
dataset = datasets.TokenizedKoMRC.load('data/train.json')
train_dataset, eval_dataset = datasets.TokenizedKoMRC.split(dataset)
print("Number of Questions", len(train_dataset), len(eval_dataset))
print(train_dataset[0])

Number of Questions 10834 1203
{'guid': '1ad2029a2cae445498692ba3e9256116', 'context_original': '지난 11월 14일 토요일, 클룩(KLOOK)은 진에어와 함께 관광 비행 및 내년 홍콩 왕복 항공권을 증정하는 ‘미리 즐기는 홍콩원정대’ 항공여행을 진행했다. 클룩은 본 여행을 위해, 진에어 전세기 운항을 주도해 실현시켰다. 특히 이번 상품은 홍콩 여행 테마를 주제로 잃어버린 여행의 설렘을 다시 느낄 수 있도록 한 관광 비행과 얼리버드 홍콩 왕복 항공권을 결합시켜 예약이 조기 매진되는 등 화제를 불러일으킨 패키지다. 오랜만의 비행을 맞이해, 클룩과 진에어 외에도 유관 기관의 협조가 빛을 발했다. 인천공항에서는 미니 콘서트와 국내선 시설 지원을, 제주공항 관제소에서는 한라산 및 제주도 전역 저공비행 허가를 밀어줬다. 여기에 더해, 홍콩관광청에서도 홍콩 여행을 필수적인 가이드북과 컬러링북 등 특전들이 참가자들에게 제공했다. 홍콩원정대 이름에 걸맞게 홍콩 명물 제니쿠키 역시 서비스되었다. 이처럼 기업과 기관이 협조해 진행한 덕분에, 저렴한 비용에 역대급 특전 제공이 가능했다. 홍콩 테마의 관광 비행 1인과 홍콩 왕복 항공권 1매, 클룩의 홍콩 여행 상품권 10만원권을 포함한 가격이 26만 9천원, 관광 비행 1인과 홍콩 왕복 항공권 2매, 클룩 홍콩 여행 상품권 15만원 포함 패키지는 39만 9천원으로 제공되었다. 관광 비행만 원하는 경우를 위해서는 클룩의 국내 여행 상품권 5만원권을 포함한 16만 9천원의 패키지가 선택 가능했다. 함께 제공되는 클룩 여행상품권은 현지 어트랙션이나 입장권은 물론 호텔과 렌터카 등 다양한 여행상품을 구매할 수 있다. 또한 홍콩 왕복 항공권과 클룩 상품권의 경우 2022년 3월까지의 넉넉한 유효기간으로 제공되어, 코로나19로 유효기간 내 사용이 불가능한 경우 유효기간 연장이 가능하다. 단, 홍콩 항공권은 사용 당시 유류할증료와 공항세만 별도 지불하면 된다. 홍콩원정

In [5]:
sample = train_dataset[0]
print(sample['context'][sample['answers'][0]['start']:sample['answers'][0]['end']+1])

['16', '만', '9', '천', '원']


In [6]:
tokenizer = utils.Tokenizer.build_vocab(dataset)
print(tokenizer.sample2ids(train_dataset[0]))

Counting Vocab: 100%|██████████| 12037/12037 [00:46<00:00, 260.70it/s]


{'guid': '1ad2029a2cae445498692ba3e9256116', 'context': '지난 11월 14일 토요일, 클룩(KLOOK)은 진에어와 함께 관광 비행 및 내년 홍콩 왕복 항공권을 증정하는 ‘미리 즐기는 홍콩원정대’ 항공여행을 진행했다. 클룩은 본 여행을 위해, 진에어 전세기 운항을 주도해 실현시켰다. 특히 이번 상품은 홍콩 여행 테마를 주제로 잃어버린 여행의 설렘을 다시 느낄 수 있도록 한 관광 비행과 얼리버드 홍콩 왕복 항공권을 결합시켜 예약이 조기 매진되는 등 화제를 불러일으킨 패키지다. 오랜만의 비행을 맞이해, 클룩과 진에어 외에도 유관 기관의 협조가 빛을 발했다. 인천공항에서는 미니 콘서트와 국내선 시설 지원을, 제주공항 관제소에서는 한라산 및 제주도 전역 저공비행 허가를 밀어줬다. 여기에 더해, 홍콩관광청에서도 홍콩 여행을 필수적인 가이드북과 컬러링북 등 특전들이 참가자들에게 제공했다. 홍콩원정대 이름에 걸맞게 홍콩 명물 제니쿠키 역시 서비스되었다. 이처럼 기업과 기관이 협조해 진행한 덕분에, 저렴한 비용에 역대급 특전 제공이 가능했다. 홍콩 테마의 관광 비행 1인과 홍콩 왕복 항공권 1매, 클룩의 홍콩 여행 상품권 10만원권을 포함한 가격이 26만 9천원, 관광 비행 1인과 홍콩 왕복 항공권 2매, 클룩 홍콩 여행 상품권 15만원 포함 패키지는 39만 9천원으로 제공되었다. 관광 비행만 원하는 경우를 위해서는 클룩의 국내 여행 상품권 5만원권을 포함한 16만 9천원의 패키지가 선택 가능했다. 함께 제공되는 클룩 여행상품권은 현지 어트랙션이나 입장권은 물론 호텔과 렌터카 등 다양한 여행상품을 구매할 수 있다. 또한 홍콩 왕복 항공권과 클룩 상품권의 경우 2022년 3월까지의 넉넉한 유효기간으로 제공되어, 코로나19로 유효기간 내 사용이 불가능한 경우 유효기간 연장이 가능하다. 단, 홍콩 항공권은 사용 당시 유류할증료와 공항세만 별도 지불하면 된다. 홍콩원정대 전세기는 오후 3시, 인천국제공항을 이륙해 광주, 제주, 부산, 대구

In [7]:
train_dataset = utils.TokenizerWrapperDataset(train_dataset, tokenizer)
eval_dataset = utils.TokenizerWrapperDataset(eval_dataset, tokenizer)

print(train_dataset[0])

{'guid': '1ad2029a2cae445498692ba3e9256116', 'context': '지난 11월 14일 토요일, 클룩(KLOOK)은 진에어와 함께 관광 비행 및 내년 홍콩 왕복 항공권을 증정하는 ‘미리 즐기는 홍콩원정대’ 항공여행을 진행했다. 클룩은 본 여행을 위해, 진에어 전세기 운항을 주도해 실현시켰다. 특히 이번 상품은 홍콩 여행 테마를 주제로 잃어버린 여행의 설렘을 다시 느낄 수 있도록 한 관광 비행과 얼리버드 홍콩 왕복 항공권을 결합시켜 예약이 조기 매진되는 등 화제를 불러일으킨 패키지다. 오랜만의 비행을 맞이해, 클룩과 진에어 외에도 유관 기관의 협조가 빛을 발했다. 인천공항에서는 미니 콘서트와 국내선 시설 지원을, 제주공항 관제소에서는 한라산 및 제주도 전역 저공비행 허가를 밀어줬다. 여기에 더해, 홍콩관광청에서도 홍콩 여행을 필수적인 가이드북과 컬러링북 등 특전들이 참가자들에게 제공했다. 홍콩원정대 이름에 걸맞게 홍콩 명물 제니쿠키 역시 서비스되었다. 이처럼 기업과 기관이 협조해 진행한 덕분에, 저렴한 비용에 역대급 특전 제공이 가능했다. 홍콩 테마의 관광 비행 1인과 홍콩 왕복 항공권 1매, 클룩의 홍콩 여행 상품권 10만원권을 포함한 가격이 26만 9천원, 관광 비행 1인과 홍콩 왕복 항공권 2매, 클룩 홍콩 여행 상품권 15만원 포함 패키지는 39만 9천원으로 제공되었다. 관광 비행만 원하는 경우를 위해서는 클룩의 국내 여행 상품권 5만원권을 포함한 16만 9천원의 패키지가 선택 가능했다. 함께 제공되는 클룩 여행상품권은 현지 어트랙션이나 입장권은 물론 호텔과 렌터카 등 다양한 여행상품을 구매할 수 있다. 또한 홍콩 왕복 항공권과 클룩 상품권의 경우 2022년 3월까지의 넉넉한 유효기간으로 제공되어, 코로나19로 유효기간 내 사용이 불가능한 경우 유효기간 연장이 가능하다. 단, 홍콩 항공권은 사용 당시 유류할증료와 공항세만 별도 지불하면 된다. 홍콩원정대 전세기는 오후 3시, 인천국제공항을 이륙해 광주, 제주, 부산, 대구

In [8]:
collator = utils.Collator(tokenizer)
train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True, collate_fn=collator, num_workers=4)
eval_loader = DataLoader(eval_dataset, batch_size=16, shuffle=False, collate_fn=collator, num_workers=4)
batch = next(iter(train_loader))
print(batch['input_ids'].shape)
print(batch['input_ids'])
print(list(batch.keys()))

torch.Size([16, 943])
tensor([[    2,   145,    30,  ...,     0,     0,     0],
        [    2,   344,    32,  ...,     0,     0,     0],
        [    2,  3331, 23956,  ...,     0,     0,     0],
        ...,
        [    2,   507,  7304,  ...,     0,     0,     0],
        [    2,  8818,   483,  ...,     0,     0,     0],
        [    2,  6089,   210,  ...,     0,     0,     0]])
['guid', 'context', 'question', 'position', 'input_ids', 'token_type_ids', 'start', 'end', 'attention_mask']


In [10]:
from transformers import BertConfig

config = BertConfig(
     vocab_size=tokenizer.vocab_size,
     max_position_embeddings=1024,
     hidden_size=512,
     num_hidden_layers=6,
     num_attention_heads=8,
     intermediate_size=2048
)
model = models.BertForQuestionAnswering(config)
model.cuda()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(36311, 512, padding_idx=0)
      (position_embeddings): Embedding(1024, 512)
      (token_type_embeddings): Embedding(2, 512)
      (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=512, out_features=512, bias=True)
              (key): Linear(in_features=512, out_features=512, bias=True)
              (value): Linear(in_features=512, out_features=512, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=512, out_features=512, bias=True)
              (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise

In [11]:
writer = SummaryWriter(log_dir='review')
os.makedirs('dump', exist_ok=True)
optimizer = torch.optim.AdamW(model.parameters(), lr=2e-4)
step = 0

In [ ]:
for epoch in range(1, 31):
    print("Epoch", epoch)
    for batch in tqdm(train_loader, desc='Train'):
        del batch['guid'], batch['context'], batch['question'], batch['position']
        batch = {key: value.cuda() for key, value in batch.items()}
        start = batch.pop('start')
        end = batch.pop('end')
        
        start_logits, end_logits = model(**batch)
        loss = F.cross_entropy(start_logits, start) + F.cross_entropy(end_logits, end)

        optimizer.zero_grad(set_to_none=True)
        loss.backward()
        clip_grad_norm_(model.parameters(), max_norm=1.)
        optimizer.step()

        step += 1
        writer.add_scalar('Train Loss', loss.item(), step)
        del batch, start, end, start_logits, end_logits, loss

    with torch.no_grad():
        losses = []
        for batch in tqdm(eval_loader, desc="Evaluation"):
            del batch['guid'], batch['context'], batch['question'], batch['position']
            batch = {key: value.cuda() for key, value in batch.items()}
            start = batch.pop('start')
            end = batch.pop('end')

            start_logits, end_logits = model(**batch)
            loss = F.cross_entropy(start_logits, start) + F.cross_entropy(end_logits, end)

            losses.append(loss.item())
            del batch, start, end, start_logits, end_logits, loss
        loss = sum(losses) / len(losses)
        writer.add_scalar('Eval Loss', loss, step)

    model.save_pretrained(f'dump/model.{epoch}')
        

In [12]:
model = models.BertForQuestionAnswering.from_pretrained('dump/model.30')
model.cuda()
model.eval()

BertForQuestionAnswering(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(36311, 512, padding_idx=0)
      (position_embeddings): Embedding(1024, 512)
      (token_type_embeddings): Embedding(2, 512)
      (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=512, out_features=512, bias=True)
              (key): Linear(in_features=512, out_features=512, bias=True)
              (value): Linear(in_features=512, out_features=512, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=512, out_features=512, bias=True)
              (LayerNorm): LayerNorm((512,), eps=1e-12, elementwise

In [13]:
for idx, raw_sample in zip(range(1, 4), train_dataset):
    sample = dict(raw_sample) 
    context = sample.pop('context')
    question = sample.pop('question')
    position = sample.pop('position')
    start, end = sample.pop('start'), sample.pop('end')
    del sample['guid']

    sample = {key: value.cuda()[None, :] for key, value in sample.items()}
    
    with torch.no_grad():
        start_logits, end_logits = model(**sample)
    start_logits.squeeze_(0), end_logits.squeeze_(0)

    print(f'------{idx}------')
    print('Context:', context)
    print('Question:', question)
    print('Answer:', tokenizer.logits2answer(raw_sample, start_logits, end_logits))

------1------
Context: 지난 11월 14일 토요일, 클룩(KLOOK)은 진에어와 함께 관광 비행 및 내년 홍콩 왕복 항공권을 증정하는 ‘미리 즐기는 홍콩원정대’ 항공여행을 진행했다. 클룩은 본 여행을 위해, 진에어 전세기 운항을 주도해 실현시켰다. 특히 이번 상품은 홍콩 여행 테마를 주제로 잃어버린 여행의 설렘을 다시 느낄 수 있도록 한 관광 비행과 얼리버드 홍콩 왕복 항공권을 결합시켜 예약이 조기 매진되는 등 화제를 불러일으킨 패키지다. 오랜만의 비행을 맞이해, 클룩과 진에어 외에도 유관 기관의 협조가 빛을 발했다. 인천공항에서는 미니 콘서트와 국내선 시설 지원을, 제주공항 관제소에서는 한라산 및 제주도 전역 저공비행 허가를 밀어줬다. 여기에 더해, 홍콩관광청에서도 홍콩 여행을 필수적인 가이드북과 컬러링북 등 특전들이 참가자들에게 제공했다. 홍콩원정대 이름에 걸맞게 홍콩 명물 제니쿠키 역시 서비스되었다. 이처럼 기업과 기관이 협조해 진행한 덕분에, 저렴한 비용에 역대급 특전 제공이 가능했다. 홍콩 테마의 관광 비행 1인과 홍콩 왕복 항공권 1매, 클룩의 홍콩 여행 상품권 10만원권을 포함한 가격이 26만 9천원, 관광 비행 1인과 홍콩 왕복 항공권 2매, 클룩 홍콩 여행 상품권 15만원 포함 패키지는 39만 9천원으로 제공되었다. 관광 비행만 원하는 경우를 위해서는 클룩의 국내 여행 상품권 5만원권을 포함한 16만 9천원의 패키지가 선택 가능했다. 함께 제공되는 클룩 여행상품권은 현지 어트랙션이나 입장권은 물론 호텔과 렌터카 등 다양한 여행상품을 구매할 수 있다. 또한 홍콩 왕복 항공권과 클룩 상품권의 경우 2022년 3월까지의 넉넉한 유효기간으로 제공되어, 코로나19로 유효기간 내 사용이 불가능한 경우 유효기간 연장이 가능하다. 단, 홍콩 항공권은 사용 당시 유류할증료와 공항세만 별도 지불하면 된다. 홍콩원정대 전세기는 오후 3시, 인천국제공항을 이륙해 광주, 제주, 부산, 대구 등을 하늘에서 둘러본 뒤 다시 인천공항으로 돌아왔다. 약 2

/usr/local/lib/python3.8/dist-packages/torch/_tensor.py:575: UserWarning: floor_divide is deprecated, and will be removed in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values.
To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor'). (Triggered internally at  ../aten/src/ATen/native/BinaryOps.cpp:467.)
  return torch.floor_divide(self, other)


In [16]:
test_dataset = datasets.TokenizedKoMRC.load('data/test.json')
test_dataset = utils.TokenizerWrapperDataset(test_dataset, tokenizer)
print("Number of Questions", len(test_dataset))
print(test_dataset[0])

Number of Questions 4008
{'guid': 'd14cb73158624cf094c546d856fd3c80', 'context': 'BMW 코리아(대표 한상윤)는 창립 25주년을 기념하는 ‘BMW 코리아 25주년 에디션’을 한정 출시한다고 밝혔다. 이번 BMW 코리아 25주년 에디션(이하 25주년 에디션)은 BMW 3시리즈와 5시리즈, 7시리즈, 8시리즈 총 4종, 6개 모델로 출시되며, BMW 클래식 모델들로 선보인 바 있는 헤리티지 컬러가 차체에 적용돼 레트로한 느낌과 신구의 조화가 어우러진 차별화된 매력을 자랑한다. 먼저 뉴 320i 및 뉴 320d 25주년 에디션은 트림에 따라 옥스포드 그린(50대 한정) 또는 마카오 블루(50대 한정) 컬러가 적용된다. 럭셔리 라인에 적용되는 옥스포드 그린은 지난 1999년 3세대 3시리즈를 통해 처음 선보인 색상으로 짙은 녹색과 풍부한 펄이 오묘한 조화를 이루는 것이 특징이다. M 스포츠 패키지 트림에 적용되는 마카오 블루는 1988년 2세대 3시리즈를 통해 처음 선보인 바 있으며, 보랏빛 감도는 컬러감이 매력이다. 뉴 520d 25주년 에디션(25대 한정)은 프로즌 브릴리언트 화이트 컬러로 출시된다. BMW가 2011년에 처음 선보인 프로즌 브릴리언트 화이트는 한층 더 환하고 깊은 색감을 자랑하며, 특히 표면을 무광으로 마감해 특별함을 더했다. 뉴 530i 25주년 에디션(25대 한정)은 뉴 3시리즈 25주년 에디션에도 적용된 마카오 블루 컬러가 조합된다. 뉴 740Li 25주년 에디션(7대 한정)에는 말라카이트 그린 다크 색상이 적용된다. 잔잔하면서도 오묘한 깊은 녹색을 발산하는 말라카이트 그린 다크는 장식재로 활용되는 광물 말라카이트에서 유래됐다. 뉴 840i xDrive 그란쿠페 25주년 에디션(8대 한정)은 인도양의 맑고 투명한 에메랄드 빛을 연상케 하는 몰디브 블루 컬러로 출시된다. 특히 몰디브 블루는 지난 1993년 1세대 8시리즈에 처음으로 적용되었던 만큼 이를 오마주하는 의미를 담고 있

In [19]:
import os
import csv

os.makedirs('out', exist_ok=True)
with torch.no_grad(), open('out/baseline.csv', 'w') as fd:
    writer = csv.writer(fd)
    writer.writerow(['Id', 'Predicted'])

    rows = []
    for raw_sample in tqdm(test_dataset, "Testing"):
        sample = dict(raw_sample) 
        guid = sample.pop('guid')
        context = sample.pop('context')
        position = sample.pop('position')
        start, end = sample.pop('start'), sample.pop('end')
        del sample['question']

        sample = {key: value.cuda()[None, :] for key, value in sample.items()}
        start_logits, end_logits = model(**sample)
        start_logits.squeeze_(0), end_logits.squeeze_(0)

        rows.append([guid, tokenizer.logits2answer(raw_sample, start_logits, end_logits)])
    
    writer.writerows(rows)

Testing: 100%|██████████| 4008/4008 [00:45<00:00, 88.38it/s] 
